<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignment2/partA/Assignment2_partA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import *
!pip install wandb
import wandb
from wandb.keras import WandbCallback 

     |████████████████████████████████| 2.1MB 6.7MB/s 
     |████████████████████████████████| 133kB 39.4MB/s 
     |████████████████████████████████| 163kB 27.5MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=0c097f2efd709e23b6e2ee7a4da853ebf142eefb3b88d24d032bc854cd38610d
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0130dfe218ff42edc4576909097bd9f5225ec71fa3c127a257f3541110b4a7c7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Splitting training data into training and validation set in the ratio 9:1 

# # import os
# # path = '/content/drive/MyDrive/nature_12K.zip'
# # import zipfile
# # with zipfile.ZipFile(path, 'rw') as zip_ref:
# #     zip_ref.extractall('/content/drive/MyDrive/Assignment2')
# !pip install split-folders tqdm
# import splitfolders
# splitfolders.ratio(path+'train', output=path+'split', seed=1337, ratio=(.9, .1), group_prefix=None) # default values

In [3]:
path = '/content/drive/MyDrive/Assignment2/inaturalist_12K/' #Load the inaturalist_12K dataset

##simple function which returns the CNN consisting of 5 convolution layers
#filters is a list , where filters[i] indicates num of filters in ith convolutional layer (0 indexing)
#kern_filter is a list of tuples denoting the kernel_size of the filter
#actiavation_fn is a list of activation functions

def simplenetwork(filters,kern_filter,activation_fn,dense_layer_size):
    model = Sequential()
    for i in range(5):
          if(i == 0):
            model.add(layers.Conv2D(filters[i],kern_filter[i],input_shape=(224,224,3),activation=activation_fn[i]))     
          else :
            model.add(layers.Conv2D(filters[i],kern_filter[i],activation=activation_fn[i]))
          model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(dense_layer_size,activation=activation_fn[5]))
    model.add(layers.Dense(10,activation='softmax'))
    return model

In [8]:
model = simplenetwork([32,32,32,32,32],[(3,3),(3,3),(3,3),(3,3),(3,3)],['relu' for i in range(6)],1024)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 52, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 32)       

In [9]:
#More general function where we can add drop_out batch_normalisation , avg_pool etc.
#filters is a list , where filters[i] indicates num of filters in ith convolutional layer (0 indexing)
#kern_filter is a list of tuples denoting the kernel_size of the filter
#actiavation_fn is a list of activation functions
#drop_out is the value of drop_out we will apply
#batch_normalisation if set true we do batch normalisation after each layer
#dense_layer is the size of dense layer
#train is train data
#validation is validation data
#num_classes is the number of output classes , this case we pass 10
#epochs is number of epochs
#drop_batch_dense , if 'both' then we apply dropout followed by batchnormalisation after dense layer , if set 'batch' we do only batch_normalisation after dense layer and if ser 'drop' we apply dropout after dense layer
#last2avg_pool if set true then for last2 Convolutional layers we do avgpool instead of maxpooling
#dp_after_each CNN , if set true then we apply dropout after every ConV layer else we apply dropout only on last2 convolutional layers

def network(filters,kern_filter,activation_fn,drop_out,batch_normalisation,dense_layer,train,validation,num_classes,epochs,drop_batch_dense,last_2_avgpool,dp_aft_eachCNN):
    
     model = Sequential()
     
     if (batch_normalisation):
        for i in range(5):
          if(i == 0):
            model.add(layers.Conv2D(filters[i],kern_filter[i],input_shape=(224,224,3),activation=activation_fn[i]))     
          else :
            model.add(layers.Conv2D(filters[i],kern_filter[i],activation=activation_fn[i]))

          if (i<=2 or not(last_2_avgpool)):
             if(dp_aft_eachCNN or i>2):
                model.add(layers.Dropout(drop_out))
             model.add(layers.BatchNormalization())
             model.add(layers.MaxPooling2D(pool_size=(2,2)))
          else:
             model.add(layers.Dropout(drop_out))
             model.add(layers.BatchNormalization())
             model.add(layers.AveragePooling2D(pool_size=(2, 2)))
     else:
        for i in range(5):
          if(i == 0):
            model.add(layers.Conv2D(filters[i],kern_filter[i],input_shape=(224,224,3),activation=activation_fn[i]))
          else :
            model.add(layers.Conv2D(filters[i],kern_filter[i],activation=activation_fn[i]))
          if(i<=2 or not(last_2_avgpool)):
             if(i>2 or dp_aft_eachCNN):
               model.add(layers.Dropout(drop_out))
             model.add(layers.MaxPooling2D(pool_size=(2,2)))
          else:
             model.add(layers.Dropout(drop_out))
             model.add(layers.AveragePooling2D(pool_size=(2, 2)))

     model.add(layers.Flatten())
     model.add(layers.Dropout(drop_out))
     model.add(layers.BatchNormalization())
     model.add(layers.Dense(dense_layer,activation='relu'))  #activation_fn[5]))
     if(drop_batch_dense == 'both' or drop_batch_dense == 'drop'):
         model.add(layers.Dropout(drop_out))
     if(drop_batch_dense == 'both' or drop_batch_dense == 'batch'):
         model.add(layers.BatchNormalization())
     model.add(layers.Dense(num_classes))
     model.add(layers.Softmax())
     labels = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
     print(model.summary())

     model.compile(optimizer='adam',loss=losses.CategoricalCrossentropy(),metrics=['accuracy'])
     model.fit(train,epochs=epochs,validation_data=validation,callbacks=[WandbCallback()])

     return model